In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.applications.resnet50 import ResNet50
from keras.models import load_model
import tensorflow as tf
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from keras import backend as K

In [ ]:
use_log_dir='X:/train-scene classification/logs_resnet50_vanilla' # Change for every new training session 

class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir=use_log_dir, **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()

In [ ]:
img_width, img_height = 150, 150
train_data_dir = "X:/train-scene classification/data/train/"
validation_data_dir = "X:/train-scene classification/data/valid/"
test_data_dir = "X:/train-scene classification/data/test/"
nb_train_samples = 16384
nb_validation_samples = 650 
batch_size = 32
epochs = 1000
# Save the model according to the conditions  
chkpt_dir = "X:/train-scene classification/chkpts/resnet50_vanilla.h5" # Change for every new training session 

In [ ]:
# model = ResNet50(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3), classes = 6)

In [ ]:
# model.save('resnet50.h5')
model = load_model('resnet50.h5')

In [ ]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
# for layer in model.layers[:200]:
#     layer.trainable = False

model.summary()
print(len(model.layers))

In [ ]:
print(model.layers[3].get_config())

In [ ]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(2048, activation="relu")(x)
x = Dropout(0.7)(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.3)(x)
predictions = Dense(6, activation="softmax")(x)

In [ ]:
# creating the final model 
model_final = Model(input = model.input, output = predictions)
model_final.summary()

## If resuming training (and the notebook hasn't been closed after the previous training session), run code from here.

In [ ]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), metrics=["accuracy"])

In [ ]:
model_final.load_weights(chkpt_dir)

In [ ]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=10)

validation_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=10)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = validation_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

In [ ]:
# Save the model according to the conditions  
# chkpt_dir = "X:/train-scene classification/chkpts/densenet_extra_fc.h5" # Change for every new training session 

checkpoint = ModelCheckpoint(chkpt_dir, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=40, verbose=1, mode='auto')
#[TrainValTensorBoard(write_graph=False)]

In [ ]:
import os

# Train the model 
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
validation_data = validation_generator,
nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early, TrainValTensorBoard(write_graph=False)])

In [ ]:
#Saving Model
model_final.save("vgg_vanilla.h5")

## Testing + Predictions

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        color_mode="rgb",
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

filenames = test_generator.filenames
nb_samples = len(filenames)

predict = model.predict_generator(test_generator,steps = nb_samples)

In [ ]:
K.set_learning_phase(0) #For setting inference phase

In [ ]:
model_final = load_model(chkpt_dir)

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
test_data_dir_2 = "X:/train-scene classification/data/test/final_test"

In [ ]:
labels = (train_generator.class_indices)

labels = dict((v,k) for k,v in labels.items())
print(labels)
# predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
train_dir = "X:/train-scene classification/data/train/0_buildings"
img = cv2.imread(os.path.join(train_dir, "146.jpg"))

# valid_dir = "X:/train-scene classification/data/valid/0_buildings"
# img = cv2.imread(os.path.join(valid_dir, "2800.jpg"))

# img = cv2.imread(os.path.join(test_data_dir_2, "398.jpg"))
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb)
print(img.shape)
img = np.reshape(img, (1, img_width, img_height, 3))
img = img / 255.
pred = np.argmax(model_final.predict(img), axis=1)
print(pred, labels[pred[0]])

In [ ]:
#### model = load_model("")

In [ ]:
# test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# test_generator = test_datagen.flow_from_directory(
#     directory="X:/train-scene classification/data/test/",
#     target_size=(img_width, img_height),
#     color_mode="rgb",
#     batch_size=1,
#     class_mode=None,
#     shuffle=False,
#     seed=42
# )

In [ ]:
# test_generator.reset()
# pred=model_final.predict_generator(test_generator,verbose=1)

In [ ]:
# import numpy as np
# predicted_class_indices=np.argmax(pred,axis=1)

In [ ]:
# print(predicted_class_indices[:20])
# print(predicted_class_indices.shape)
# print(pred.shape)

In [ ]:
import pandas as pd
df = pd.read_csv(os.path.join("X:/train-scene classification", 'test_images.csv'))
print(df.head())
test_list = df.image_name.values
print(test_list[:20])

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
join_l = []
ans = []
test_data_dir_2 = "X:/train-scene classification/data/test/final_test"
for i, im in enumerate(test_list):
    img = cv2.imread(os.path.join(test_data_dir_2, im))
    if i == 1:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img_rgb)
    img = cv2.resize(img, (img_width, img_height)) 
    img = np.reshape(img, (1, img_width, img_height, 3))
    img = img / 255.
    pred = np.argmax(model_final.predict(img), axis=1)
    ans.append((im, pred))

# x_test = np.vstack(join_l)
# print(x_test.shape)

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
join_l = []
ans = []
test_data_dir_2 = "X:/train-scene classification/data/test/final_test"
'''
for i, im in enumerate(test_list):
    img = cv2.imread(os.path.join(test_data_dir_2, im))
    img = cv2.resize(img, (img_width, img_height)) 
    img = np.reshape(img, (1, img_width, img_height, 3))
    pred = np.argmax(model_final.predict(img), axis=1)
    ans.append((im, pred))
'''

for i in range(0, len(test_list), 32):
    j = i
    batch = []
    for j in range(i, i+32):
        if(j>=len(test_list)):
            break
        img = cv2.imread(os.path.join(test_data_dir_2, test_list[j]))
        img = cv2.resize(img, (img_width, img_height)) 
        img = np.reshape(img, (1, img_width, img_height, 3))
        batch.append(img)
    batch = np.vstack(batch)
    pred = np.argmax(model_final.predict(batch), axis=1)
    for j in range(i, i+32):
        if(j>=len(test_list)):
            break
        ans.append((test_list[j], pred[j-i]))
print(ans[:10])
# x_test = np.vstack(join_l)
# print(x_test.shape)

In [ ]:
li = []
for i in ans:
    pa = i[0]
    val= i[1][0]
    li.append([pa, val])
print(li[:10])

In [ ]:
res_df = pd.DataFrame(li, columns=["image_name", "label"])
res_df.head()

In [ ]:
res_df.to_csv('submission_resnet50_corrected_normalized.csv', index = False)

In [ ]:
model_final.evaluate()